In [1]:
import pandas as pd
import gdown
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
import os
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from google.colab import files
import shutil
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

#PP Data for fine-tuning

In [ ]:
srouce = 'https://drive.google.com/uc?id='


ids = {
    # chuong a
    '1vJyvAG7rFY2Di8Q4uJ5IRdFHhshCMeC6':'1_a.txt',
    '1xm3cuTcmPC-8tjS_9lOJLQXZmJSPVJxQ':'2_a.txt',
    # chuong b
    '16uTNbpZC3zqFH5k7yqN5PiqESiQPTynS':'1_b.txt',
    '19CeUoOsJYqgDrLf21MZjlG8fjf8rh3yw':'2_b.txt',
    # chuong c
    '16uTNbpZC3zqFH5k7yqN5PiqESiQPTynS':'1_c.txt',
    '19CeUoOsJYqgDrLf21MZjlG8fjf8rh3yw':'2_c.txt',
    # chuong d
    '1cYLNcqTvewioe3nsj3fj5-ampE8TwwZZ':'1_d.txt',
    # chuong f
    '1-Y0tcd-ZArRyeGnPfbprynM2UA7yni4E':'1_f.txt',
    '1Es-98x--B5CtRwhjhe2-a9zPOJO_p8pJ':'2_f.txt',
    '1JUe0JCug0O5DB2DtHIBLb1-rTKlIjNTm':'3_f.txt',
    '1A9TrbnpbM_6-xT68050OygNN7vqWgQRu':'4_f.txt',
    '1Bv756PIe3efQNAlDET8vaUP3hxkGRKks':'5_f.txt',
    '1WjLC5S-S9BZU1H_9QYKXhYIvdj0dgJ0j':'6_f.txt',
    '1hd_PGTJBl0gfaI6qr7tRRsJMzBekuJhd':'7_f.txt',
    '1Bs1luYvdoe4qIsil8K3YPlkxj5TCAmLO':'8_f.txt',
    '10NLbv47Cqmf8rv_aU9-QH_QJTikIg-9F':'9_f.txt',
    '161z8im5cZzlLXAnMnBIC-QMVqwooHNXc':'10_f.txt',
    '149fnn8ZvP0ToJ0s5okXWwkAexP2qzKc5':'11_f.txt',
    '1gS9J1NdB4HzFIRrgCz2MeEZmJm0nd561':'12_f.txt',

    #chuong g
    '1mx6mh7wosCV2gmQss1DZErQ6jpBk9_Vk':'1_g.txt',
    '1JNaI66roTYPqENU-3wPMGcg-YGU0gMv1':'2_g.txt',
    '1qlrQYKGyClIdC-H1J-OjnlqSgdxcypo-':'3_g.txt',

    #chuong b cs
    '1_hW1LwoHaX1H_TdXK4XoLvU1GWYLepiP':'1_h.txt',
    '1ML2Lkuvn5rlgGjqQPSHzQ2U8hAUb6TfN':'2_h.txt',
    '1dBttwyuSXapubCibfV--l6Qt27NjAOY5':'3_h.txt',
    '1eQ2qDtXR8dxGbHs9JD_PeC-GeLKv5N2t':'4_h.txt',

}


for key, value in ids.items():
    gdown.download(srouce+key, value, quiet=True)

KeyboardInterrupt: 

In [ ]:
def sort_key(filename):
    chapter = filename.split('_')[1].split('.')[0]
    sequence = int(filename.split('_')[0])
    return (chapter, sequence)

sorted_files = sorted(ids.values(), key=sort_key)

directory_path = '/content'

# Write sorted files into the output file
with open('sgk_12.txt', 'w') as collection:
    for file_name in sorted_files:
        path = os.path.join(directory_path, file_name)
        if os.path.isfile(path):
            with open(path, 'r') as f:
                collection.write(f.read())
                collection.write("\n")

#Already have it


In [6]:
with open('sgk_12.txt', 'r') as f:
    data = f.read()
data

'\ufeffBài 1 GIỚI THIỆU VỀ TRÍ TUỆ NHÂN TẠO\n\nHọc xong bài này, em sẽ:\n\nGiải thích được sơ lược về khái niệm Trí tuệ nhân tạo (AI).\n\n Nêu được ví dụ để thấy một hệ thống AI có thức, có khả năng suy luận và khả năng học...\nBiết được một số lĩnh vực nghiên cứu của AI.\n\n1) Theo em, máy tính làm được những việc gì tốt hơn con người?\n2) Em hãy cho ví dụ một việc mà máy tính chưa làm được tốt hơn con người.\n\n1. Khái niệm Trí tuệ nhân tạo (AI)\n\nTheo em, khi máy tính làm được những gì thỉ ta nói máy tính có trí tuệ?\n\na) Trí tuệ con người và Trí tuệ nhân tạo.\n\nCon người có khả năng tư duy, suy luận logic, phân tích và giải quyết vấn đề, biết học\nhỏi và rút kinh nghiệm từ những sai lầm trước đó, biết kiểm soát và điều chỉnh cảm xúc của\nbản thân,... Những khả năng đó thể hiện trí tuệ con người.\n\nMáy tính tự động xử lý khối lượng lớn dữ liệu rất nhanh và chính xác. Vấn để được\nđặt ra là: liệu có cách nào để làm cho máy tính suy nghĩ và hành động như con người\nđược không? Để 

In [7]:
def remove_multi_spaces(text):
    return re.sub(r' +', ' ', text)

def remove_brackets(text):
    return re.sub(r'\((.*?)\)', lambda L: '', text)

def remove_square_brackets(text):
    return re.sub(r'\[(.*?)\]', lambda L: '', text)

def remove_multi_downline(text):
    return re.sub('\n+','\n',text)

In [8]:
data = remove_multi_downline(data)
data = remove_multi_spaces(data)
data = data[1:] if data.startswith('\ufeff') else data

In [9]:
data

'Bài 1 GIỚI THIỆU VỀ TRÍ TUỆ NHÂN TẠO\nHọc xong bài này, em sẽ:\nGiải thích được sơ lược về khái niệm Trí tuệ nhân tạo (AI).\n Nêu được ví dụ để thấy một hệ thống AI có thức, có khả năng suy luận và khả năng học...\nBiết được một số lĩnh vực nghiên cứu của AI.\n1) Theo em, máy tính làm được những việc gì tốt hơn con người?\n2) Em hãy cho ví dụ một việc mà máy tính chưa làm được tốt hơn con người.\n1. Khái niệm Trí tuệ nhân tạo (AI)\nTheo em, khi máy tính làm được những gì thỉ ta nói máy tính có trí tuệ?\na) Trí tuệ con người và Trí tuệ nhân tạo.\nCon người có khả năng tư duy, suy luận logic, phân tích và giải quyết vấn đề, biết học\nhỏi và rút kinh nghiệm từ những sai lầm trước đó, biết kiểm soát và điều chỉnh cảm xúc của\nbản thân,... Những khả năng đó thể hiện trí tuệ con người.\nMáy tính tự động xử lý khối lượng lớn dữ liệu rất nhanh và chính xác. Vấn để được\nđặt ra là: liệu có cách nào để làm cho máy tính suy nghĩ và hành động như con người\nđược không? Để trả lời cho câu hỏi đó, 

In [10]:
tokenizer = AutoTokenizer.from_pretrained("NlpHUST/gpt2-vietnamese")
model = AutoModelForCausalLM.from_pretrained("NlpHUST/gpt2-vietnamese")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/854k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/512k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [11]:
max_length = 512

input_sequences = []

for line in data.lower().split('\n'):
    tokens = tokenizer.encode(line)
    for i in range(1, len(tokens)):
        n_gram_tokens = tokens[:i+1]
        if len(n_gram_tokens) <= max_length:
            n_gram_text = tokenizer.decode(n_gram_tokens)
            input_sequences.append(n_gram_text)
        else:
            for j in range(0, len(n_gram_tokens), max_length):
                chunk_tokens = n_gram_tokens[j:j+max_length]
                chunk_text = tokenizer.decode(chunk_tokens)
                input_sequences.append(chunk_text)

In [8]:
input_sequences[1:10]

['bài 1 giới',
 'bài 1 giới thiệu',
 'bài 1 giới thiệu về',
 'bài 1 giới thiệu về trí',
 'bài 1 giới thiệu về trí tuệ',
 'bài 1 giới thiệu về trí tuệ nhân',
 'bài 1 giới thiệu về trí tuệ nhân tạo',
 'học xong',
 'học xong bài']

In [56]:
features = []
labels = []
tokenizer.padding_side = 'left'
tokenizer.add_special_tokens({'pad_token': '<s>'})
for sequence in input_sequences:
    tokens = tokenizer.encode(sequence,
                                  max_length=max_length,
                                  padding='max_length',
                                  truncation=True,
                                  return_tensors="pt")
    input_encoding = tokens[:, :-1]
    target_encoding = tokens[:, -1]

    features.append(input_encoding)
    labels.append(target_encoding)




In [47]:
len(features), len(labels)

(36072, 36072)

In [49]:
tokenizer.decode(labels[1])

' giới'

In [19]:
print(tokenizer)

GPT2TokenizerFast(name_or_path='NlpHUST/gpt2-vietnamese', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<eos>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50257: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, specia

In [27]:
features[0] , labels[0]


(tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [ ]:
input_sequences[1:10]

['bài 1 giới',
 'bài 1 giới thiệu',
 'bài 1 giới thiệu về',
 'bài 1 giới thiệu về trí',
 'bài 1 giới thiệu về trí tuệ',
 'bài 1 giới thiệu về trí tuệ nhân',
 'bài 1 giới thiệu về trí tuệ nhân tạo',
 'học xong',
 'học xong bài']

In [57]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return {
            'input_ids': self.features[idx],
            'labels': self.labels[idx]
        }


In [ ]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)


transformer.wte.weight False
transformer.wpe.weight False
transformer.h.0.ln_1.weight False
transformer.h.0.ln_1.bias False
transformer.h.0.attn.c_attn.weight False
transformer.h.0.attn.c_attn.bias False
transformer.h.0.attn.c_proj.weight False
transformer.h.0.attn.c_proj.bias False
transformer.h.0.ln_2.weight False
transformer.h.0.ln_2.bias False
transformer.h.0.mlp.c_fc.weight False
transformer.h.0.mlp.c_fc.bias False
transformer.h.0.mlp.c_proj.weight False
transformer.h.0.mlp.c_proj.bias False
transformer.h.1.ln_1.weight False
transformer.h.1.ln_1.bias False
transformer.h.1.attn.c_attn.weight False
transformer.h.1.attn.c_attn.bias False
transformer.h.1.attn.c_proj.weight False
transformer.h.1.attn.c_proj.bias False
transformer.h.1.ln_2.weight False
transformer.h.1.ln_2.bias False
transformer.h.1.mlp.c_fc.weight False
transformer.h.1.mlp.c_fc.bias False
transformer.h.1.mlp.c_proj.weight False
transformer.h.1.mlp.c_proj.bias False
transformer.h.2.ln_1.weight False
transformer.h.2.ln_1

In [15]:
for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
    if 'ln_f' in name:
        param.requires_grad = True

for name, param in model.named_parameters():
    print(name, param.requires_grad)


transformer.wte.weight False
transformer.wpe.weight False
transformer.h.0.ln_1.weight False
transformer.h.0.ln_1.bias False
transformer.h.0.attn.c_attn.weight False
transformer.h.0.attn.c_attn.bias False
transformer.h.0.attn.c_proj.weight False
transformer.h.0.attn.c_proj.bias False
transformer.h.0.ln_2.weight False
transformer.h.0.ln_2.bias False
transformer.h.0.mlp.c_fc.weight False
transformer.h.0.mlp.c_fc.bias False
transformer.h.0.mlp.c_proj.weight False
transformer.h.0.mlp.c_proj.bias False
transformer.h.1.ln_1.weight False
transformer.h.1.ln_1.bias False
transformer.h.1.attn.c_attn.weight False
transformer.h.1.attn.c_attn.bias False
transformer.h.1.attn.c_proj.weight False
transformer.h.1.attn.c_proj.bias False
transformer.h.1.ln_2.weight False
transformer.h.1.ln_2.bias False
transformer.h.1.mlp.c_fc.weight False
transformer.h.1.mlp.c_fc.bias False
transformer.h.1.mlp.c_proj.weight False
transformer.h.1.mlp.c_proj.bias False
transformer.h.2.ln_1.weight False
transformer.h.2.ln_1

In [ ]:
dataset = CustomDataset(features, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

optimizer = AdamW(model.parameters(), lr=1e-5)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit='batch'):
        inputs = batch['input_ids']
        targets = batch['labels']

        attention_mask = (inputs != tokenizer.pad_token_id).float()
        optimizer.zero_grad()
        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()
        total += targets.size(0)

    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    model_save_path = f"model_epoch_{epoch+1}.pt"
    tokenizer_save_path = f"tokenizer_epoch_{epoch+1}"

    torch.save(model.state_dict(), model_save_path)
    tokenizer.save_pretrained(tokenizer_save_path)
    # print(f"Saved model to {model_save_path} and tokenizer to {tokenizer_save_path}")


Epoch 1/10:   0%|          | 0/1128 [00:00<?, ?batch/s]